# DSTM Climbing Club 

### Testing lesson


This notebook walks you through writing a query and testing it.

This notebook assumes you already have an auth.json file in the current directory, where this notebook is located.

#About the Data

* **Peak(name, elev, diff, map, region)**:
    Information about the mountain peaks: name; elevation; difficulty; which map it's on; region of the mountain range

* **Climber(name, sex)**: CC membership list; includes name and gender
* **Participated(trip_id, name)**:   Contains the set of climbers who participated in each CC sponsored trip
* **Climbed(trip_id, peak, when_climbed)**: Lists which peaks were climbed on each trip and the trip date

In [ ]:
%load_ext sql

The cells below create a connection string that reads your local auth.json file, returns a string that allows you to connect to the database, and actually connects you to your database.

In [ ]:
import json
def make_conn_str():
    with open("auth.json", "r") as f:
        auth = json.load(f)
        
    username = auth["username"]
    password = auth["password"]
    hostname = "orion-196-39.crc.rice.edu"
    db = username
    
    return f"postgresql://{username}:{password}@{hostname}/{db}"

In [ ]:
conn_str = make_conn_str()

In [ ]:
%sql $conn_str 

Which climbers have not made it to the top of a difficulty 5 peak?

First, take a peek at the climbers

In [ ]:
%%sql
SELECT * 
FROM CLIMBER 
ORDER BY RANDOM() 
LIMIT 10;

In [ ]:
%%sql
SELECT * 
FROM PARTICIPATED
ORDER BY RANDOM()
LIMIT 10

In [ ]:
%%sql
SELECT * 
FROM CLIMBED
ORDER BY RANDOM()
LIMIT 10

Why include RANDOM()?

and the peaks

In [ ]:
%%sql
SELECT * 
FROM PEAK 
ORDER BY region, map
LIMIT 50;

What are the different difficulty values?

In [ ]:
%%sql 
SELECT MIN(diff), MAX(diff)
FROM PEAK;

In [ ]:
%%sql
SELECT diff, COUNT(diff)
FROM PEAK
GROUP BY diff
ORDER BY diff;

Great. Now we know what the data look like. So, next we want to start breaking down the question. Let's tackle the "difficulty 5 peak" first.

This looks pretty straight-forward from our **peak** table. 

In [ ]:
%%sql
SELECT * 
FROM PEAK
WHERE diff = 5;

It's easy to see in the results, that diff = 5. If the criteria were more complex, 

That query gives us all the diffculty 5 peaks. Do we need all the attributes?


In [ ]:
%%sql
SELECT name 
FROM PEAK
WHERE diff = 5;

How can we relate climbers to peaks?

Through **CLIMBED** and **PARTICIPATED**

In [ ]:
%%sql
SELECT *
FROM CLIMBED d,PEAK k, PARTICIPATED p
WHERE d.TRIP_ID = p.TRIP_ID
    AND d.PEAK = k.NAME
LIMIT 10;

What does this query give us?

Everyone who ever climbed a peak and when they climbed it.

Let's check it. What attributes would we expect?

From **CLIMBED**: trip_id, peak, when_climbed
    
From **CLIMBER**:name, sex

From **PEAK**: name, elev, diff, map, region

From **PARTICIPATED**: trip_id, name

Attributes look good. Does it return the right answer?

Let's use a subset of our data to check.

You want to test a number of conditions:

#1. When there is a match between one relation and another
#2. When there isn't a match between the two relations and another
#3. When a record in one relation matches more than 1 record in another

So, in this case, let's empty out **CLIMBER**, **CLIMBED**, and **PEAK** and build a small set of records.

In [ ]:
%%sql

DELETE FROM CLIMBED;

DELETE FROM CLIMBER;

DELETE FROM PARTICIPATED;

DELETE FROM PEAK;

-- pick a set of records with different data
INSERT INTO CLIMBER VALUES ('SANDRA','F');
INSERT INTO CLIMBER VALUES ('DONNA','F');
INSERT INTO CLIMBER VALUES ('JOHN','M');

INSERT INTO PEAK VALUES ('Kern Point',12730,2,'Mt Kaweah','Kaweahs and West');

INSERT INTO PARTICIPATED VALUES (9, 'DONNA');
INSERT INTO PARTICIPATED VALUES (9, 'JOHN');

INSERT INTO CLIMBED VALUES (9,'Kern Point','2002-09-16');


Now, if I rerun my query, I would expect to see that Donna and John both climbed Kern Point, but Sandra didn't.

## It is super important to know what answer you expect BEFORE you run the query

Go back and rerun the query!

Back to our original question: 

Which climbers have not made it to the top of a difficulty 5 peak?

Which climbers have?

In [ ]:
%%sql
SELECT *
FROM CLIMBED d,PEAK k, PARTICIPATED p
WHERE d.TRIP_ID = p.TRIP_ID
    AND d.PEAK = k.NAME
    AND k.diff = 5;

No one!

That's because our limited dataset didn't include any level 5 peaks. So, let's add **one** to the trip.



In [ ]:
%%sql
-- NOTE: Changing the difficulty to 5 from the original data
INSERT INTO PEAK VALUES ('Mount Mallory',13845,5,'Mount Whitney','Corocoran to Whitney');

INSERT INTO CLIMBED VALUES (9,'Mount Mallory','2002-09-17');


and rerun the query

This time I see my two climbers.

Let's make this last query (which climbers have climbed a difficulty 5 peak) into a VIEW, so we can reuse it:

In [ ]:
%%sql
CREATE VIEW diff5Climbers AS
(
    SELECT *
    FROM CLIMBED d,PEAK k, PARTICIPATED p
    WHERE d.TRIP_ID = p.TRIP_ID
        AND d.PEAK = k.NAME
        AND k.diff = 5
)

Hmm, that didn't work. Let's fix it

In [ ]:
%%sql
DROP VIEW IF EXISTS diff5Climbers;
CREATE VIEW diff5Climbers AS
(
    SELECT p.NAME, k.NAME peakName
    FROM CLIMBED d,PEAK k, PARTICIPATED p
    WHERE d.TRIP_ID = p.TRIP_ID
        AND d.PEAK = k.NAME
        AND k.diff = 5
)

and now we can use the VIEW

In [ ]:
%%sql
SELECT * 
FROM diff5Climbers;

Now, let's finish answering our question:

Which climbers have not made it to the top of a difficulty 5 peak?

Again, given our data, what results should we see?



In [ ]:
%%sql
SELECT NAME 
FROM CLIMBER
    EXCEPT
    -- Climbers who have made it to the top of a difficulty 5 peak
        SELECT NAME 
        FROM diff5Climbers
ORDER BY NAME;


Finally, don't forget to restore the original data!

In [ ]:
%run ./ClimbingClubData.ipynb

In [ ]:
%%sql
SELECT NAME 
FROM CLIMBER
    EXCEPT
    -- Climbers who have made it to the top of a difficulty 5 peak
        SELECT NAME 
        FROM diff5Climbers
ORDER BY NAME;


## Lessons Learned / to Learn

* Don't try to look at all of the data
* Look at samples of the data
* Look at extremes of the data
* Reduce the data
* Change the data
    * But only change one things at a time
* Try ALL possible combinations
* Try to check the result a different way 
    * Remeber checking subtraction with addition
* Add abstractions (e.g. VIEWs)
* Don't forget to restore the original data!

Copyright ©2019 Christopher M Jermaine (cmj4@rice.edu), and Risa B Myers  (rbm2@rice.edu)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.